In [ ]:
to_inf_file = '/root/autodl-tmp/2022.04.06(正式数据)/附件3/无位置信息的图片汇总表.csv'



import numpy as np
import pandas as pd



df = pd.read_csv(to_inf_file, encoding='gbk')

image_names = df['文件名'].unique()

data_img_dir = '/root/autodl-tmp/2022.04.06(正式数据)/附件1/'
dest_img_dir = 'mmdetection/data/image2test'

import os, shutil
'''
for image_name in image_names:
    src = os.path.join(data_img_dir, image_name)
    dst = os.path.join(dest_img_dir, image_name)
    shutil.copyfile(src, dst)
'''

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

config_file = 'mmdetection/configs/yolox/my_yolox_v7_pre70_200e.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
# url: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
checkpoint_file = 'mmdetection/work_dirs/my_yolox_v7_pre70_200e/best_bbox_mAP_epoch_75.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

img = 'mmdetection/data/image2test/00002.jpg'
result = inference_detector(model, img)

In [ ]:
thr = 0.3

In [17]:
class_bbox_candidates = []


for i in range(0, len(result)):
    for bbox_candi in result[i]:
        if bbox_candi[4] > 0.3:
            x1 = bbox_candi[0]
            y1 = bbox_candi[1]
            x2 = bbox_candi[2]
            y2 = bbox_candi[3]
            bbox_class = i
            score = bbox_candi[4]
            class_bbox_candidate = {
                'x1': x1,
                'y1': y1,
                'x2': x2,
                'y2': y2,
                'bbox_class': bbox_class,
                'score': score
            }
            class_bbox_candidates.append(class_bbox_candidate)
            

In [66]:
class_bbox_candidates = []
thr = 0.7

for image_name in image_names:
    
    img = os.path.join(dest_img_dir, image_name)
    result = inference_detector(model, img)
    for i in range(0, len(result)):
        for bbox_candi in result[i]:
            if bbox_candi[4] > thr:
                x1 = bbox_candi[0]
                y1 = bbox_candi[1]
                x2 = bbox_candi[2]
                y2 = bbox_candi[3]
                bbox_class = i
                score = bbox_candi[4]

                class_bbox_candidate = {
                    'x1': x1,
                    'y1': y1,
                    'x2': x2,
                    'y2': y2,
                    'bbox_class': bbox_class,
                    'score': score,
                    'image_name': image_name
                }
                class_bbox_candidates.append(class_bbox_candidate)
    
    

In [67]:
len(class_bbox_candidates)

331

In [49]:
result_df = pd.DataFrame(columns=['序号','文件名','虫子编号','中心点x坐标','中心点y坐标','左上角x坐标','左上角y坐标','右下角x坐标','右下角y坐标'])
result_df

,序号,文件名,虫子编号,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标


In [26]:
# 原始标注目录
original_ann_dir = '/root/autodl-tmp/2022.04.06(正式数据)/附件2/'
# 原始标注文件
original_ann = os.path.join(original_ann_dir, '图片虫子位置详情表.csv')

ori_df = pd.read_csv(original_ann, encoding='gbk')
ori_df

,序号,文件名,虫子编号,虫子名称,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标
0,1,00001.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00004.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
2,3,00006.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00008.jpg,7,二化螟,1984.0,2150.0,1835.0,1920.0,2134.0,2380.0
4,5,00008.jpg,228,二点委夜蛾,3301.0,827.0,3040.0,618.0,3563.0,1037.0
...,...,...,...,...,...,...,...,...,...,...
2651,2652,03811.jpg,0,无,NaN,NaN,NaN,NaN,NaN,NaN
2652,2653,03812.jpg,430,豆野螟,1441.0,2270.0,1178.0,2130.0,1703.0,2411.0
2653,2654,03813.jpg,10,褐飞虱属,2310.0,623.0,2277.0,566.0,2343.0,680.0
2654,2655,03813.jpg,280,黄毒蛾,3070.0,217.0,2878.0,70.0,3262.0,365.0


In [29]:
class_name_list = ['无', '二化螟', '二点委夜蛾', '棉铃虫', '褐飞虱属', '白背飞虱', '八点灰灯蛾', '蝼蛄', '蟋蟀', '甜菜夜蛾', '黄足猎蝽', '稻纵卷叶螟', '甜菜白带野螟', '黄毒蛾', '石蛾', '大黑鳃金龟', '粘虫', '稻螟蛉', '甘蓝夜蛾', '地老虎', '大螟', '瓜绢野螟', '线委夜蛾', '水螟蛾', '紫条尺蛾', '歧角螟', '草地螟', '豆野螟', '干纹冬夜蛾']
name_dic = {v:k for v,k in enumerate(class_name_list)}


In [33]:
name_dic[2]

'二点委夜蛾'

In [43]:
ori_df[ori_df['虫子名称'] == name_dic[0]]
ori_df['虫子名称'] == name_dic[0]
ori_df[ori_df['虫子名称'] == name_dic[28]]['虫子编号'].unique()[0]

673

In [45]:
continuousIndex2officialIndex = []
for i in range(0,29):
    continuousIndex2officialIndex.append(ori_df[ori_df['虫子名称'] == name_dic[i]]['虫子编号'].unique()[0])

In [46]:
continuousIndex2officialIndex

[0,
 7,
 228,
 222,
 10,
 9,
 156,
 41,
 256,
 115,
 148,
 8,
 398,
 280,
 480,
 485,
 105,
 310,
 235,
 25,
 6,
 402,
 394,
 392,
 387,
 401,
 110,
 430,
 673]

In [68]:
result_df = pd.DataFrame(columns=['序号','文件名','虫子编号','中心点x坐标','中心点y坐标','左上角x坐标','左上角y坐标','右下角x坐标','右下角y坐标'])

for i, candi in enumerate(class_bbox_candidates):
    medium_coor_x = int((candi['x1']+candi['x2'])/2)
    medium_coor_y = int((candi['y1']+candi['y2'])/2)

    row = {
        '序号' : i,
        '文件名': candi['image_name'],
        '虫子编号': continuousIndex2officialIndex[candi['bbox_class']],
        '中心点x坐标': medium_coor_x,
        '中心点y坐标': medium_coor_y,
        '左上角x坐标': int(candi['x1']),
        '左上角y坐标': int(candi['y1']),
        '右下角x坐标': int(candi['x2']),
        '右下角y坐标': int(candi['y2'])
    }
    result_df = result_df.append(row, ignore_index=True)

/tmp/ipykernel_200616/1976042418.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
/tmp/ipykernel_200616/1976042418.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
/tmp/ipykernel_200616/1976042418.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
/tmp/ipykernel_200616/1976042418.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
/tmp/ipykernel_200616/1976042418.py:18: FutureWarning: The frame.append method is deprec

In [69]:
result_df

,序号,文件名,虫子编号,中心点x坐标,中心点y坐标,左上角x坐标,左上角y坐标,右下角x坐标,右下角y坐标
0,0,00015.jpg,156,3588,43,3464,0,3712,88
1,1,00018.jpg,480,4444,546,4283,299,4605,793
2,2,00021.jpg,156,1208,2165,827,1751,1590,2579
3,3,00028.jpg,156,2760,3384,2401,3120,3119,3648
4,4,00028.jpg,156,3913,2161,3565,1795,4260,2526
...,...,...,...,...,...,...,...,...,...
326,326,03814.jpg,156,4442,181,4094,0,4790,362
327,327,03814.jpg,256,2063,3505,1963,3361,2163,3649
328,328,03815.jpg,156,3055,2201,2665,1831,3445,2570
329,329,03815.jpg,156,3739,2938,3535,2618,3943,3257


In [70]:
result_df.to_csv('result2.csv', index=False, encoding='gbk')

In [71]:

result3_df = pd.DataFrame(columns=['序号','文件名','虫子编号','数量'])
result3_df

,序号,文件名,虫子编号,数量


In [91]:
result3_df = pd.DataFrame(columns=['序号','文件名','虫子编号','数量'])

insect_count = 0
for index, row in result_df.iterrows():
    if index == 0:
        fileName = row['文件名']
        insectNo = row['虫子编号']
    if fileName == row['文件名'] and insectNo == row['虫子编号']:
        update = False
        insect_count += 1
    else:
        fileName = row['文件名']
        insectNo = row['虫子编号']
        insect_count += 1
        result3_row = {
        '序号': index,
        '文件名': row['文件名'],
        '虫子编号': row['虫子编号'],
        '数量': insect_count
        }
        result3_df = result3_df.append(result3_row, ignore_index=True)
        insect_count = 0




/tmp/ipykernel_200616/918723957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result3_df = result3_df.append(result3_row, ignore_index=True)
/tmp/ipykernel_200616/918723957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result3_df = result3_df.append(result3_row, ignore_index=True)
/tmp/ipykernel_200616/918723957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result3_df = result3_df.append(result3_row, ignore_index=True)
/tmp/ipykernel_200616/918723957.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result3_df = result3_df.append(result3_row, ignore_index=True)
/tmp/ipykernel_200616/918723957.py:21: FutureWarning

In [92]:
result3_df

,序号,文件名,虫子编号,数量
0,1,00018.jpg,480,2
1,2,00021.jpg,156,1
2,3,00028.jpg,156,1
3,8,00033.jpg,156,5
4,9,00053.jpg,392,1
...,...,...,...,...
250,324,03792.jpg,156,1
251,326,03814.jpg,156,2
252,327,03814.jpg,256,1
253,328,03815.jpg,156,1


In [93]:
result3_df.to_csv('result3.csv', index=False, encoding='gbk')

In [ ]:
class_bbox_candidates

In [98]:
result3_df = pd.DataFrame(columns=['序号','文件名','虫子编号','数量'])

result3_df = result_df.groupby(['文件名','虫子编号']).count


In [108]:
result_df['文件名']

0      00015.jpg
1      00018.jpg
2      00021.jpg
3      00028.jpg
4      00028.jpg
         ...    
326    03814.jpg
327    03814.jpg
328    03815.jpg
329    03815.jpg
330    03815.jpg
Name: 文件名, Length: 331, dtype: object

In [105]:
result_df[['文件名','虫子编号']].nunique()

文件名     212
虫子编号     16
dtype: int64

In [109]:
result3_df = result_df.groupby(['文件名','虫子编号']).size().reset_index(name='虫子数量')

In [110]:
result3_df

,文件名,虫子编号,虫子数量
0,00015.jpg,156,1
1,00018.jpg,480,1
2,00021.jpg,156,1
3,00028.jpg,156,5
4,00033.jpg,156,1
...,...,...,...
251,03792.jpg,156,2
252,03814.jpg,156,1
253,03814.jpg,256,1
254,03815.jpg,156,2


In [ ]:
result3_df.to_csv('result3.csv', index=False, encoding='gbk')

In [99]:
result3_df

<bound method GroupBy.count of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f09451eac70>>

In [ ]:
result[1][0]